In [ ]:
%autocall
import streamlit as st
import os,sys
import pandas as pd
import folium
# from streamlit_folium import st_folium
from datetime import datetime
import string
import re
from folium.plugins import MarkerCluster
import numpy as np
import requests
import glob
import json
from pathlib import Path

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
read_files = glob.glob("./data/2022/zip_geojs/*.json")                                                                                                                                                                         
out = []                                                                                                                                                                                           

for f in read_files: 
    # /print f
    fn = Path(f).stem
    # /print fn[:2]
    with open(f, "rb") as infile:
        out.append((fn[:2].upper(), json.load(infile)))                                                                                                                                                                           

out = dict(out)

In [ ]:
!pwd

In [ ]:
df = pd.read_csv('./data/states.csv')
df.dropna()
df.loc[len(df.index)] = ['VI','Idaho','44.068202','-114.742041']
df

In [ ]:
pac = pd.read_csv('./data/2022/processed_pac2022.csv')
# receipts = pd.read_csv('./data/2022/schedule_a-2023-11-06T22_05_26.csv', dtype='object')

In [ ]:
receipts['committee_state']=''
for row_idx, id in enumerate(receipts['committee_id'].tolist()):
    if id in pac['committee_id'].tolist():
        col_idx=list(receipts.columns).index('committee_state')
        p_col_index = list(pac.columns).index('committee_state')
        receipts.iloc[row_idx,col_idx] = pac.iloc[pac['committee_id'].tolist().index(id),p_col_index]

In [ ]:
receipts['lat'] = ''
receipts['lon'] = ''

for n, i in enumerate(receipts.iterrows()):
    idx = list(receipts.columns).index('committee_state')
    if receipts.iloc[n,idx] in df['state'].to_list():
            receipts.iloc[n,list(receipts.columns).index('lat')] = float(df.iloc[df['state'].to_list().index(receipts.iloc[n,idx]), 2])
            receipts.iloc[n,list(receipts.columns).index('lon')] = float(df.iloc[df['state'].to_list().index(receipts.iloc[n,idx]), 3])
            receipts.iloc[n,idx] = df.iloc[df['state'].to_list().index(receipts.iloc[n,idx]), 1]
    else:
            receipts.iloc[n,list(receipts.columns).index('lat')] = 0
            receipts.iloc[n,list(receipts.columns).index('lon')] = 0


In [ ]:
pac['lat'] = ''
pac['lon'] = ''

for n, i in enumerate(pac.iterrows()):
    idx = list(pac.columns).index('committee_state')
    if pac.iloc[n,idx] in df['state'].to_list():
            pac.iloc[n,list(pac.columns).index('lat')] = float(df.iloc[df['state'].to_list().index(pac.iloc[n,idx]), 2])
            pac.iloc[n,list(pac.columns).index('lon')] = float(df.iloc[df['state'].to_list().index(pac.iloc[n,idx]), 3])
            pac.iloc[n,idx] = df.iloc[df['state'].to_list().index(pac.iloc[n,idx]), 1]
    else:
            pac.iloc[n,list(pac.columns).index('lat')] = 0
            pac.iloc[n,list(pac.columns).index('lon')] = 0


In [ ]:
receipts = receipts.drop(columns=['committee_name.1','contributor_street_2','receipt_type_full','memo_code','memo_code_full'])
receipts = receipts.drop(columns=['original_sub_id','back_reference_transaction_id','back_reference_schedule_name'])
receipts = receipts.drop(columns=['candidate_first_name', 'candidate_last_name', 'candidate_middle_name','candidate_prefix','candidate_suffix','candidate_office', 'candidate_office_full','candidate_office_state','candidate_office_state_full','candidate_office_district','conduit_committee_id','conduit_committee_name','conduit_committee_street1','conduit_committee_street2','conduit_committee_city','conduit_committee_state','conduit_committee_zip'])

In [ ]:
receipts.to_csv('./data/2022/processed_schedule_a2022.csv')

In [ ]:
callback = """\
function (row) {
    var icon, marker;
    icon = L.AwesomeMarkers.icon({
        icon: "map-marker", markerColor: "blue"});
    marker = L.marker(new L.LatLng(row[0], row[1]));
    marker.setIcon(icon);
    return marker;
};
"""

In [ ]:
receipts['contributor_aggregate_ytd']=receipts['contributor_aggregate_ytd'].apply(lambda x: float(x))
receipts[['committee_name','committee_state','contributor_aggregate_ytd']].groupby(['committee_name','committee_state']).sum()



In [ ]:
# receipts[['committee_name','lat','lon']]
pac[['committee_state','receipts']]
pac['receipts'] = pac['receipts'].apply(lambda x : float(x))
receipts_agg = pac[['committee_state','receipts']].groupby(['committee_state']).sum().reset_index(['committee_state'])

In [ ]:
pac.to_csv('./data/2022/processed_pac2022.csv')

In [ ]:
m = folium.Map(location=[48, -96.5], zoom_start=4.5, scrollWheelZoom=False, tiles='CartoDB positron')

choropleth = folium.Choropleth(
    geo_data='./data/us-state-boundaries.geojson',
    name="choropleth",
    data=candidates,
    columns=["committee_state", "receipts"],
    key_on="feature.properties.name",
    fill_color="YlGn",
    line_opacity=0.2,
    fill_opacity=0.7,
    highlight=True,
    legend_name="Total PAC Receipts By State, YTD 2022"
)

state_list=['']
for n, feature in enumerate(choropleth.geojson.data['features']):
    state_name = feature['properties']['state']
    state_list.append(state_name)
    feature['properties']['receipts'] = 'PACs aggregated receipts ytd: ' + '{:,}'.format(receipts_agg.iloc[n, 1]) if state_name in receipts_agg['committee_state'].tolist() else ''

choropleth.geojson.add_to(m)


mc = MarkerCluster(name='PACs Geolocations').add_to(m)


for i in range(0,len(pac)):
    if pac.iloc[i]['committee_id'] in receipts['committee_id'].tolist():
        if type(pac.iloc[i]['lat']) != str:
            folium.Marker(
                location=[pac.iloc[i]['lat'], pac.iloc[i]['lon']],
                popup=pac.iloc[i]['committee_name'],
            callback=callback).add_to(mc)
        

    
folium.LayerControl().add_to(m)


m

In [ ]:
columns = pd.read_csv("./data/2022/weball-fields.csv")
fields = columns["Field name"].tolist()

In [ ]:
fields

In [ ]:
candidates = pd.read_csv("./data/2022/weball22.txt", sep="|",header=None, encoding='latin-1')
candidates.columns = fields

In [ ]:
candidates

In [ ]:
for i in candidates.columns:
    if ('contribution' or 'from' or 'loan') in i.lower():
        print(i)

In [ ]:
for i in candidates.columns:
    if 're' in i.lower() and 'receipt' not in i.lower():
        print(i)

In [ ]:
contributions = ["Contributions from candidate",
"Total individual contributions",
"Contributions from party committees",
"Contributions from other political committees","Transfers from authorized committees","Loans from candidate","Other loans"]
contributions.insert(0, 'Candidate name')


In [ ]:
#convert to float

def func_float(df):
    return df.apply(lambda x: float(x))

for i in contributions:
    if i != 'Candidate name':
        candidates[i] = func_float(candidates[i])
    else:
        pass

In [ ]:
candidates['Total contributions'] = candidates[contributions[1:]].agg(['sum'],axis=1)

In [ ]:
candidates

In [ ]:
refunds = ["Candidate loan repayments",
"Other loan repayments",
"Refunds to individuals",
"Refunds to committees"]
refunds.insert(0, 'Candidate name')

candidates[refunds]

In [ ]:
#convert to float

def func_float(df):
    return df.apply(lambda x: float(x))

for i in refunds:
    if i != 'Candidate name':
        candidates[i] = func_float(candidates[i])
    else:
        pass

In [ ]:
candidates['Total refunds'] = candidates[refunds[1:]].agg(['sum'],axis=1)

In [ ]:
candidates['Net contributions'] = candidates['Total contributions'] - candidates['Total refunds']

In [24]:
candidates.to_csv("./data/2022/processed_weball22.csv")
candidates = pd.read_csv("./data/2022/processed_weball22.csv")

In [ ]:
fields = pd.read_csv("./data/2022/cands_committee[fields].csv")
ccl = pd.read_csv("./data/2022/cands-committee[data].txt", sep="|", header=None)
ccl

In [ ]:
candidates['Affiliated Committee Id'] = ""
candidates['Affiliated Committee Type'] = ""
for n , i in enumerate(candidates['Candidate identification'].tolist()):
    if i in ccl.iloc[:,0].tolist():
        candidates.iloc[n,-2] = ccl.iloc[ccl.iloc[:,0].tolist().index(i),3]
        candidates.iloc[n,-1] = ccl.iloc[ccl.iloc[:,0].tolist().index(i),4]


In [ ]:
candidates

In [ ]:
import numpy as np

In [ ]:
committees = pd.read_csv("./data/2022/committee-master[data].txt",sep="|",header=None)
fields = pd.read_csv("./data/2022/committee_master[fields].csv")
fields = [i for i in fields['Field name'].tolist() if str(i) != 'nan']
committees.columns = fields

In [ ]:
candidates[['Affiliated Committee Name','Street one',
 'Street two',
 'City or town',
 'State',
 'ZIP code',]] = ""

In [ ]:
committees

In [ ]:
for i in candidates['Affiliated Committee Id'].tolist():
    if i in committees['Committee identification'].tolist():
        candidates.iloc[candidates['Affiliated Committee Id'].tolist().index(i), -6] = committees.iloc[ committees['Committee identification'].tolist().index(i),1]
        candidates.iloc[candidates['Affiliated Committee Id'].tolist().index(i), -5] = committees.iloc[ committees['Committee identification'].tolist().index(i),3]
        candidates.iloc[candidates['Affiliated Committee Id'].tolist().index(i), -4] = committees.iloc[ committees['Committee identification'].tolist().index(i),4]
        candidates.iloc[candidates['Affiliated Committee Id'].tolist().index(i), -3] = committees.iloc[ committees['Committee identification'].tolist().index(i),5]
        candidates.iloc[candidates['Affiliated Committee Id'].tolist().index(i), -2] = committees.iloc[ committees['Committee identification'].tolist().index(i),6]
        candidates.iloc[candidates['Affiliated Committee Id'].tolist().index(i), -1] = committees.iloc[ committees['Committee identification'].tolist().index(i),7]

In [26]:
candidates = candidates.drop('Unnamed: 0', axis=1)
candidates

Candidate identification                   Candidate name Incumbent challenger status  Party code Party affiliation  Total receipts  Transfers from authorized committees  Total disbursements  Transfers to authorized committees  Beginning cash  Ending cash  Contributions from candidate  Loans from candidate  Other loans  Candidate loan repayments  Other loan repayments  Debts owed by  Total individual contributions Candidate state  Candidate district  Special election status  Primary election status  Runoff election status  General election status  General election percentage  Contributions from other political committees  Contributions from party committees Coverage end date  Refunds to individuals  Refunds to committees Affiliated Committee Id Affiliated Committee Type Affiliated Committee Name             Street one Street two City or town State  ZIP code        lat         lon
0                   H2AK00200       CONSTANT, CHRISTOPHER[DEM]                           C           1               DEM       164637.90                                  0.00            164637.90                                 0.0            0.00         0.00                        614.85                   0.0          0.0                        0.0                    0.0      143180.09                       158023.05              AK                 0.0                      NaN                      NaN                     NaN                      NaN                          NaN                                        1000.00                               5000.0        12/31/2022                 8300.00                   0.00               C00805580                         H     CONSTANT FOR CONGRESS          PO BOX 240361        NaN    ANCHORAGE    AK   99524.0  63.588753 -154.493062
1                   H2AK01158               PELTOLA, MARY[DEM]                           I           1               DEM      7751293.39                             186868.19           7060033.09                                 0.0            0.00    691260.30                         25.00                   0.0          0.0                        0.0                    0.0           0.00                      7149826.02              AK                 0.0                      NaN                      NaN                     NaN                      NaN                          NaN                                      384020.59                              10000.0        12/31/2022               136657.70                3912.66               C00812388                         H   MARY PELTOLA FOR ALASKA          PO BOX 210014        NaN    ANCHORAGE    AK   99521.0  63.588753 -154.493062
2                   H2AK01240                WOOL, ADAM L[DEM]                           O           1               DEM        16217.07                                  0.00             16217.07                                 0.0            0.00         0.00                       1100.00                   0.0          0.0                        0.0                    0.0           0.00                        15117.00              AK                 0.0                      NaN                      NaN                     NaN                      NaN                          NaN                                           0.00                                  0.0        07/15/2022                    0.00                   0.00               C00814053                         H    ADAM WOOL FOR CONGRESS  3991 PARKS RIDGE ROAD        NaN    FAIRBANKS    AK   99709.0  64.877203 -148.148517
3                   H2AK00218          REVAK, JOSHUA CARL[REP]                           O           2               REP       121841.00                                  0.00            121841.00                                 0.0            0.00         0.00                          0.00                   0.0          0.0                        0.0                    0.0           0.00                       

In [ ]:
candidates['ZIP code'] = candidates['ZIP code'].apply(lambda x: str(x)[:5])

In [ ]:
out['AK']['features'][0]['properties'] 

In [ ]:
zips=[]
for j in df['state']:
    if j in out.keys():
        for i in out[j]['features']:
            zips.append((i["properties"]['ZCTA5CE10'],(i["properties"]['INTPTLAT10'],i["properties"]['INTPTLON10'])))


In [ ]:
zips = dict(zips)
zips = pd.DataFrame(zips).T
zips = zips.reset_index()
zips.columns = ['ZIP code','LAT','LON']

In [ ]:
len(candidates)

In [ ]:
# candidates['lat'] = float(0)
# candidates['lon'] = float(0)

for n, i in enumerate(candidates['State']):        
    idx = list(candidates.columns).index('State')
    if candidates.iloc[n,idx] in df['state'].to_list() and candidates.iloc[n,list(candidates.columns).index('lat')] == float(0):
            candidates.iloc[n,list(candidates.columns).index('lat')] = float(df.iloc[df['state'].to_list().index(candidates.iloc[n,idx]), 2])
            candidates.iloc[n,list(candidates.columns).index('lon')] = float(df.iloc[df['state'].to_list().index(candidates.iloc[n,idx]), 3])
    # else:
    #         candidates.iloc[n,list(candidates.columns).index('lat')] = 0
    #         candidates.iloc[n,list(candidates.columns).index('lon')] = 0


In [ ]:
candidates

In [ ]:
indv = pd.read_csv("./data/2022/itcont_2022_20211214_20220103.txt", sep='|',header=None,dtype='object')
fields = pd.read_csv("./data/2022/indiv_ctrbn[fields].csv")
fields = [i for i in fields['Field name'].tolist() if str(i) != 'nan']
fields
indv.columns = fields
indv['ZIP code'] = indv['ZIP code'].apply(lambda x: str(x)[:5] if str(x) != 'nan' and not str(x).isalpha() else 0)        
indv['State']

In [ ]:
for i in indv['State']:
     if i in out.keys():
         /print i

In [28]:
candidates.to_csv("./data/2022/procesed_weball22.csv")

In [ ]:
candidates

latLon = candidates[['Party code','Party affiliation','Affiliated Committee Name','Total receipts','Total disbursements','lat','lon']]
latLon = [tuple(i[1:]) for i in latLon.itertuples()]
latLon

import folium
from folium import Tooltip
m = folium.Map(location=[39.712183, -104.998424], zoom_start=5)

colors = ['blue', 'red', 'grey']

# point_layer name list
all_gp = []
for x in range(len(latLon)):
    v = latLon[x][1]
    if latLon[x][0] == int(3):
        v = '3RD'
    pg = (latLon[x][0],v)
    all_gp.append(pg)

# Create point_layer object
unique_gp = list(set(all_gp))
vlist = []
for i,k in enumerate(unique_gp):
    locals()[f'point_layer{i}'] = folium.FeatureGroup(name=k[1])
    vlist.append(locals()[f'point_layer{i}'])

# Creating list for point_layer
pl_group = []
for n in all_gp:
    for v in vlist: 
        if n[1] == vars(v)['layer_name']:
            pl_group.append(v)

for n, ((code,pty,name,r,s,lat,lng),pg) in enumerate(zip(latLon, pl_group)):
    if r - s > 100000000:
        radius = 40
        weight = 12
    elif r - s > 1000000:
        radius = 30
        weight = 8
    elif r - s > 100000:
        radius = 20
        weight = 6
    elif r - s > 10000:
        radius = 10
        weight = 4
    elif r - s > 5000:
        radius = 4
        weight = 2
    elif r - s > 1000:
        radius = 2
        weight = 1
    else:
        radius = 1
        weight = 1
    iframe = folium.IFrame("Committee Name: "+ str(name) + "<br>" + "Election cycle: 2022" + "<br>" + "Total Raised (YTD2022): $" + str(r) + "<br>" + "Total Spent (YTD2022): $" + str(s))
    folium.CircleMarker(location=[lat, lng], radius=radius,weight=0,
        popup=folium.Popup(iframe, min_width=320, max_width=320, min_height=50,max_height=100), 
        tooltip="Name: " + str(name) + " Lat: " + str(lat) + " , Long: " + str(lng),
        fill=True,  # Set fill to True
        color=colors[int(code)-1],
        fill_opacity=0.5,line_opacity=0.3).add_to(pg)
    pg.add_to(m)

m.add_child(folium.LayerControl(collapsed=False))  
m.save("./data/2022/Map1.html")
m